In [1]:
pip install silence-tensorflow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
# train dataset
dataset_train_path = 'pp_photos-dupl_v0/dataset'

In [4]:
pip install keras-facenet

In [1]:
from numpy import load
import numpy as  np
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
pd.set_option('display.max_rows', 2200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from shutil import copyfile
import string
import random
import os

## Importing necessary libraries

In [2]:
import tensorflow as tf
print(tf.__version__)

2.9.0-dev20220203


In [3]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

## Face Extraction and compute embeddings for images using FaceNet

In [4]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet

embedder = FaceNet()

In [5]:
train_images_path = []
trainy =[]
def FaceNetEmbedding(directory,threshold=0.95):
    embedding = []
    for subdir in listdir(directory):
        path = directory + '/' + subdir
        trainy.append(subdir)
        train_images_path.append(path)
        #print(path)
        embedding.append(embedder.extract(path,threshold=threshold)[0]['embedding'])
        print(f'Embedding Created for {subdir}')
    return np.array(embedding)

In [10]:
features_dataset = FaceNetEmbedding(dataset_train_path,0.95)

1/1 [==============================] - 4s 4s/step
Embedding Created for 1.PNG
1/1 [==============================] - 0s 98ms/step
Embedding Created for 10.PNG
1/1 [==============================] - 0s 102ms/step
Embedding Created for 11.PNG
1/1 [==============================] - 0s 92ms/step
Embedding Created for 12.PNG
1/1 [==============================] - 0s 106ms/step
Embedding Created for 13.PNG
1/1 [==============================] - 0s 116ms/step
Embedding Created for 14.PNG
1/1 [==============================] - 0s 108ms/step
Embedding Created for 15.PNG
1/1 [==============================] - 0s 92ms/step
Embedding Created for 16.PNG
1/1 [==============================] - 0s 100ms/step
Embedding Created for 17.PNG
1/1 [==============================] - 0s 109ms/step
Embedding Created for 18.PNG
1/1 [==============================] - 0s 90ms/step
Embedding Created for 19.PNG
1/1 [==============================] - 0s 102ms/step
Embedding Created for 2.PNG
1/1 [====================

1/1 [==============================] - 0s 107ms/step
Embedding Created for 21.PNG
1/1 [==============================] - 0s 99ms/step
Embedding Created for 22.PNG
1/1 [==============================] - 0s 108ms/step
Embedding Created for 23.PNG
1/1 [==============================] - 0s 110ms/step
Embedding Created for 24.PNG
1/1 [==============================] - 0s 100ms/step
Embedding Created for 25.PNG
1/1 [==============================] - 0s 96ms/step
Embedding Created for 26.PNG
1/1 [==============================] - 0s 107ms/step
Embedding Created for 27.PNG
1/1 [==============================] - 0s 99ms/step
Embedding Created for 28.PNG
1/1 [==============================] - 0s 100ms/step
Embedding Created for 29.PNG
1/1 [==============================] - 0s 108ms/step
Embedding Created for 3.PNG
1/1 [==============================] - 0s 106ms/step
Embedding Created for 30.PNG
1/1 [==============================] - 0s 100ms/step
Embedding Created for 4.PNG
1/1 [================

1/1 [==============================] - 0s 100ms/step
Embedding Created for 7.PNG
1/1 [==============================] - 0s 92ms/step
Embedding Created for 8.PNG
1/1 [==============================] - 0s 112ms/step
Embedding Created for 9.PNG


In [11]:
# save arrays to one file in compressed format
savez_compressed('./faces-train_embeddings.npz', features_dataset, trainy, train_images_path)

## Finding Duplicate Images

In [12]:
#load dataset
data = load('./faces-train_embeddings.npz')
trainX_features, trainy, train_images_path = data['arr_0'], data['arr_1'], data['arr_2'] #, data['arr_3']

In [13]:
def get_duplicate_images_df(feature, matching_cutoff_threshold = 0.8):
    img_dup = pd.DataFrame([],columns=['image_id','Image_label','Image_path','Duplicate_image','Duplicate_path','similarity_score'])
    for id1,feature1 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
        for id2,feature2 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
            if id2>id1:
                image_1,label_1,path_1 = feature1[0],feature1[1],feature1[2]
                image_2,label_2,path_2 = feature2[0],feature2[1],feature2[2]

                similarity_score = cosine_similarity(image_1.reshape(1, -1),
                                                     image_2.reshape(1, -1))
                if similarity_score > matching_cutoff_threshold:
                    img_dup.loc[len(img_dup)]= [id1,label_1,path_1,label_2,path_2,similarity_score[0][0]]
    return img_dup

In [14]:
df_dup = get_duplicate_images_df(data)
df_dup

,image_id,Image_label,Image_path,Duplicate_image,Duplicate_path,similarity_score
0,6,14.PNG,pp_photos-dupl_v0/dataset/14.PNG,3.PNG,pp_photos-dupl_v0/dataset/3.PNG,0.901237


In [15]:
#export results to .csv file
df_dup.to_csv('Duplicate_images.csv', index = None, header=True)

## **Building Gradio App**

In [16]:
pip install gradio

In [21]:
import gradio as gr

In [16]:
train_images_path = []
trainy =[]

def get_duplicate_images(directory, matching_cutoff_threshold = 0.8, threshold=0.95):

    embedding = []
    for subdir in listdir(directory):
        path = directory + '/' + subdir
        trainy.append(subdir)
        train_images_path.append(path)
        embedding.append(embedder.extract(path,threshold=threshold)[0]['embedding'])
    
    features_dataset = np.array(embedding)

    savez_compressed('./faces-train_embeddings.npz', features_dataset, trainy, train_images_path)

    feature = load('./faces-train_embeddings.npz')
    img_dup = pd.DataFrame([],columns=['image_id','Image_label','Image_path','Duplicate_image','Duplicate_path','similarity_score'])
    for id1,feature1 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
        for id2,feature2 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
            if id2>id1:
                image_1,label_1,path_1 = feature1[0],feature1[1],feature1[2]
                image_2,label_2,path_2 = feature2[0],feature2[1],feature2[2]

                similarity_score = cosine_similarity(image_1.reshape(1, -1),
                                                     image_2.reshape(1, -1))
                if similarity_score > matching_cutoff_threshold:
                    img_dup.loc[len(img_dup)]= [id1,label_1,path_1,label_2,path_2,similarity_score[0][0]]
    return img_dup

In [18]:
result = get_duplicate_images('pp_photos-dupl_v0/dataset')

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 102ms/step


In [19]:
result

,image_id,Image_label,Image_path,Duplicate_image,Duplicate_path,similarity_score
0,6,15.PNG,pp_photos-dupl_v0/dataset/15.PNG,30.PNG,pp_photos-dupl_v0/dataset/30.PNG,0.901237


In [22]:
face = gr.Interface(
    get_duplicate_images, 
    [
     gr.inputs.Textbox(lines = 2, placeholder = "please enter input directory path...."),
    ],

    gr.outputs.Dataframe(),

)
face.launch()


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x2376942dfd0>,
 'http://127.0.0.1:7860/',
 None)

1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 189ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 106ms/step
